In [1]:
import requests
import pandas as pd
# from bs4 import BeautifulSoup
from selenium import webdriver
import os
import re

In [2]:
url = 'https://www.streetcheck.co.uk/postcode/startingwith/e17'

In [3]:
# r = requests.get(url)

In [4]:
# r.status_code
# returns 403 error which means The HTTP 403 Forbidden client error status response code indicates that the server understood the request but refuses to authorize it. 

In [5]:
# os.path.abspath(os.curdir)

In [6]:
# os.path.dirname(os.path.abspath(os.curdir))

In [7]:
# os.listdir(os.path.dirname(os.path.abspath(os.curdir)))

In [8]:
# want to iterate through several root files if to find  
# Development folder which has the chrome driver in it 

is_folder = False
# dir_list = None
start = os.path.dirname(os.path.abspath(os.curdir))
chrome_driver_path= None
while not is_folder:
    if 'Development' in os.listdir(start):
        filepath=start
        is_folder=True
#         print(start)
#         print(os.listdir(start))
#         print(os.path.join(start,'Development'))
        chrome_driver_path = os.path.join(start,'Development', 'chromedriver')
#         print(chrome_driver_path)
        if 'chromedriver' in os.listdir(os.path.dirname(chrome_driver_path)):
            print('chrome driver found')        
    else:
        start=os.path.dirname(start)

chrome driver found


In [39]:
driver = webdriver.Chrome(executable_path=chrome_driver_path)

In [40]:
driver.get(url)

In [41]:
addresses = driver.find_element_by_id("main_inner")

In [12]:
address_list = []
for i in addresses.find_elements_by_tag_name('a'):
    address_list.append((i.text, i.get_attribute('href')))
#     print(i.text)
#     print(i.get_attribute('href'))

In [34]:
# case sensitive, i.e. postcode must be upper case and must also contain a space
postcode_regex = '([A-Z]{1,2}[0-9R][0-9A-Z]? [0-9][A-Z]{2})'

In [27]:
re.search(postcode_regex, 'N18 2AH ggbfhdd')

<re.Match object; span=(0, 7), match='N18 2AH'>

In [13]:
# address_list[:10]

In [42]:
# TODO: Find a way to get all the text that shows "no longer in use"? might be able to 
# eliminate old addresses by using the postcodes.io API

addresses_list
for num in range(0,10):
    for i in addresses.find_elements_by_tag_name('p'):
        print(i.text)
        for j in i.find_elements_by_tag_name('a'):
            print(j.text, j.get_attribute('href'))
    #     address_list.append((i.text, i.get_attribute('href')))
    #     print(i.text)
    #     print(i.get_attribute('href'))

Below are all of the active postcodes that we currently have on file for the E17 postcode district. The list also includes expired postcodes that we may have data for (those that have been discontinued by Royal Mail). We maintain several years of data on expired codes. Click here to view demographic information about E17 as a whole.
Click here to view demographic information about E17 as a whole https://www.streetcheck.co.uk/postcodedistrict/e17
Note: Some postcodes may be listed as "Large User" - this indicates that they are assigned to large businesses and similar entities, rather than residential homes. While we often have data on the areas in which these postcodes exist, some are not tied to a particular place, so data on these may be more limited.


KeyboardInterrupt: 

In [ ]:
# addr = driver.find_element_by_class_name('')

In [38]:
# driver.close() # this will close a specific tab
driver.quit() # this will close the entire browser